<a href="https://colab.research.google.com/github/yj5x/AI-tasks/blob/main/task6ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# ========== 1. المكتبات المطلوبة ==========
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping




# ========== 2. تحميل البيانات وتجهيزها ==========

from tensorflow.keras.preprocessing.image import ImageDataGenerator


from google.colab import drive
drive.mount('/content/drive')
train_dir = '/content/drive/My Drive/chest_xray/train'
validation_dir = '/content/drive/My Drive/chest_xray/val'
val_dir = '/content/drive/My Drive/chest_xray/val'
test_dir = '/content/drive/My Drive/chest_xray/test'



# ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


# الجنيريتورز
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary') # 'binary' للتصنيف الثنائي

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)


# فئات التصنيف
classes = ['NORMAL', 'PNEUMONIA']


if not os.path.exists(train_dir):
    print(f"Error: Train directory not found - {train_dir}")
if not os.path.exists(val_dir):
    print(f"Error: Validation directory not found - {val_dir}")
if not os.path.exists(test_dir):
    print(f"Error: Test directory not found - {test_dir}")

# Check if generators found any data
if train_generator.samples == 0 or validation_generator.samples == 0 or test_generator.samples == 0:
    print("\nNo data found ")
elif validation_generator.samples < validation_generator.batch_size:
     print(f"\nWarning: Validation set is too small ({validation_generator.samples} images) for the chosen batch size ({validation_generator.batch_size})")



# ========== 5. بناء النموذج ==========
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# تجميد
base_model.trainable = False

# إضافة طبقات التصنيف
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# تجميع النموذج
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# ========== 6. تدريب النموذج ==========
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size, # Use steps_per_epoch from generator
    epochs=5,
    validation_data=validation_generator, # Use validation_generator
    validation_steps=validation_generator.samples // validation_generator.batch_size, # Use validation_steps from generator
    callbacks=[early_stopping, checkpoint]
)

# ========== 7. ضبط دقيق للنموذج (Fine-tuning) ==========
base_model.trainable = True

# إعادة تجميع النموذج مع معدل تعلم أصغر
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size, # Use steps_per_epoch from generator
    epochs=10,
    validation_data=validation_generator, # Use validation_generator
    validation_steps=validation_generator.samples // validation_generator.batch_size, # Use validation_steps from generator
    callbacks=[early_stopping, checkpoint]
)

# ========== 8. تقييم النموذج ==========
# تحميل أفضل أوزان
model.load_weights('best_model.h5')

# تقييم النموذج على جميع المجموعات - Use generators for evaluation
def evaluate_model(model, generator, set_name):
    results = model.evaluate(generator, steps=generator.samples // generator.batch_size, verbose=0) # Use steps from generator
    print(f"\n{set_name} Evaluation:")
    print(f"Loss: {results[0]:.4f}")
    print(f"Accuracy: {results[1]:.4f}")
    print('-' * 50)
    return results

# Use generators for evaluation
train_results = evaluate_model(model, train_generator, "Train")
val_results = evaluate_model(model, validation_generator, "Validation")
test_results = evaluate_model(model, test_generator, "Test")

# التنبؤات - Use generator for prediction
y_pred_prob = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size, verbose=0)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

# Get true labels from test generator
test_labels = test_generator.classes[:len(y_pred)] # Get labels for the predicted samples

# ========== 9. تصور النتائج ==========
# 9.1 أداء النموذج خلال التدريب
plt.figure(figsize=(15, 5))

# Combine histories for plotting
total_loss = history.history['loss'] + history_fine.history['loss']
total_val_loss = history.history['val_loss'] + history_fine.history['val_loss']
total_accuracy = history.history['accuracy'] + history_fine.history['accuracy']
total_val_accuracy = history.history['val_accuracy'] + history_fine.history['val_accuracy']


plt.subplot(1, 2, 1)
plt.plot(total_loss, label='Training Loss')
plt.plot(total_val_loss, label='Validation Loss')
plt.title('Training and Validation Loss', fontweight='bold')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(total_accuracy, label='Training Accuracy')
plt.plot(total_val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy', fontweight='bold')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# 9.2  الماتركس
plt.figure(figsize=(8, 6))
conf_mat = confusion_matrix(test_labels, y_pred) # Use test_labels
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes,
            yticklabels=classes)
plt.title('Confusion Matrix', fontweight='bold', fontsize=16)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# 9.3 تقرير
print("\n Detailed Classification Report:")
print(classification_report(test_labels, y_pred, target_names=classes)) # Use test_labels
print(f" Final Test Accuracy: {accuracy_score(test_labels, y_pred):.4f}") # Use test_labels




In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(test_labels, y_pred_prob) # Use test_labels and y_pred_prob for ROC curve
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()